# Imported Library

In [90]:
import psycopg2
from psycopg2 import sql
import psycopg2.extras as extras
import pandas as pd
import json
from datetime import datetime,timezone
from dateutil import tz

import os
import sys 

from configupdater import ConfigUpdater
# pip install ConfigUpdater

from dotenv import dotenv_values

from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest
from google.oauth2 import service_account

# [metadata]
# pmr_pm_plan = 2019-01-01 00:00:00
# pmr_pm_item = 2019-01-01 00:00:00
# pmr_project = 2019-01-01 00:00:00
# pmr_inventory  = 2019-01-01 00:00:00


In [91]:
is_py=False
view_name = "pmr_pm_item"
isFirstLoad=False
if is_py:
    press_Y=''
    ok=False

    if len(sys.argv) > 1:
        view_name=sys.argv[1]
    else:
        print("Enter the following input: ")
        view_name = input("View Table Name : ")
print(f"View name to load to BQ :{view_name}")

View name to load to BQ :pmr_pm_item


# Imported date

In [92]:
dt_imported=datetime.now(timezone.utc) # utc
dt_imported=datetime.strptime(dt_imported.strftime("%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M:%S")
print(f"UTC: {dt_imported}")



UTC: 2023-12-30 06:21:12


# Set view

In [93]:
log = "models_logging_change"
if view_name == "pmr_pm_plan":
    content_id = 36
    view_name_id = "pm_id"

elif view_name == "pmr_pm_item":
    content_id = 37
    view_name_id = "pm_item_id"

elif view_name == "pmr_project":
    content_id = 7
    view_name_id = "project_id"

elif view_name == "pmr_inventory":
    content_id = 14
    view_name_id = "inventory_id"

else:
    raise Exception("No specified content type id")

# Set data and cofig path

In [94]:
updater = ConfigUpdater()
updater.read(".cfg")

env_path='.env'
config = dotenv_values(dotenv_path=env_path)

In [95]:
projectId=config['PROJECT_ID']  # smart-data-ml  or kku-intern-dataai or ponthorn
credential_file=config['PROJECT_CREDENTIAL_FILE']
# C:\Windows\smart-data-ml-91b6f6204773.json
# C:\Windows\kku-intern-dataai-a5449aee8483.json
# C:\Windows\pongthorn-5decdc5124f5.json


dataset_id='SMartData_Temp'  # 'SMartData_Temp'  'PMReport_Temp'
main_dataset_id='SMartDataAnalytics'  # ='SMartDataAnalytics'  'PMReport_Main'

In [96]:
credentials = service_account.Credentials.from_service_account_file(credential_file)

table_name=view_name.replace("pmr_","temp_") #can change in ("name") to temp table
table_id = f"{projectId}.{dataset_id}.{table_name}"
print(table_id)


main_table_name=view_name.replace("pmr_","")
main_table_id = f"{projectId}.{main_dataset_id}.{main_table_name}"
print(main_table_id)

# https://cloud.google.com/bigquery/docs/reference/rest/v2/Job
to_bq_mode="WRITE_EMPTY"


client = bigquery.Client(credentials= credentials,project=projectId)

pongthorn.SMartData_Temp.temp_pm_item
pongthorn.SMartDataAnalytics.pm_item


Read Configuration File and Initialize BQ Object

In [97]:
last_imported=datetime.strptime(updater["metadata"][view_name].value,"%Y-%m-%d %H:%M:%S")
print(f"UTC:{last_imported}")

# local_zone = tz.tzlocal()
# last_imported = last_imported.astimezone(local_zone)
# print(f"Local Asia/Bangkok:{last_imported}")

UTC:2023-12-30 06:15:43


# Postgres &BigQuery

In [98]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [99]:
def get_bq_table():
 try:
    table=client.get_table(table_id)  # Make an API request.
    print("Table {} already exists.".format(table_id))
    print(table.schema)
    return True
 except NotFound:
    raise Exception("Table {} is not found.".format(table_id))
    
def collectBQError(x_job):
 if x_job.errors is not None:
    for error in x_job.errors:  
      msg=f"{error['reason']} - {error['message']}"
      listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,msg])
    if   len(listError)>0:
     logErrorMessage(listError,False)  

    
def insertDataFrameToBQ(df_trasns):
    try:
        job_config = bigquery.LoadJobConfig(write_disposition=to_bq_mode,)
        job = client.load_table_from_dataframe(df_trasns, table_id, job_config=job_config)
        try:
         job.result()  # Wait for the job to complete.
        except ClientError as e:
         print(job.errors)

        print("Total ", len(df_trasns), f"Imported data to {table_id} on bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        print(e) 

# Check whether it is the first loading?

In [100]:
print("If the main table is empty , so the action of each row  must be 'added' on temp table")
rows_iter   = client.list_rows(main_table_id, max_results=1) 
no_main=len(list(rows_iter))
if no_main==0:
 isFirstLoad=True
 print(f"This is the first loaing , so there is No DATA in {main_table_id}, we load all rows from {view_name} to import into {table_id} action will be 'added' ")

If the main table is empty , so the action of each row  must be 'added' on temp table


# For The next Load
* get data from model log based on condition last_imported and table
* Get all actions from log table by selecting unique object_id and setting by doing something as logic
* Create  id and action dataframe form filtered rows from log table

In [101]:
def list_model_log(x_last_imported,x_content_id):
    sql_log = f"""
    SELECT object_id, action,TO_CHAR(date_created,'YYYY-MM-DD HH24:MI:SS') as date_created FROM {log}
    WHERE date_created  AT time zone 'utc' >= '{x_last_imported}' AND content_type_id = {x_content_id} 
    ORDER BY object_id, date_created
    """
    print(sql_log)


    # Asia/Bangkok 
    lf = list_data(sql_log, None, get_postgres_conn())
    print(f"Retrieve all rows after {last_imported}")
    print(lf.info())
    return lf


In [102]:
def select_actual_action(lf):
    listIDs=lf["object_id"].unique().tolist()
    listUpdateData=[]
    for id in listIDs:
        lfTemp=lf.query("object_id==@id")
        print(lfTemp)
        print("----------------------------------------------------------------")


        first_row = lfTemp.iloc[0]
        last_row = lfTemp.iloc[-1]
        # print(first_row)
        # print(last_row)

        if len(lfTemp)==1:
            listUpdateData.append([id,first_row["action"]])
        else:
            if first_row["action"] == "added" and last_row["action"] == "deleted":
                continue
            elif first_row["action"] == "added" and last_row["action"] != "deleted":
                listUpdateData.append([id,"added"])
            else : listUpdateData.append([id,last_row["action"]])

    print("Convert listUpdate to dataframe")
    dfUpdateData = pd.DataFrame(listUpdateData, columns= ['id', 'action'])
    dfUpdateData['id'] = dfUpdateData['id'].astype('int64')
    dfUpdateData=dfUpdateData.sort_values(by="id")
    dfUpdateData=dfUpdateData.reset_index(drop=True)

    return dfUpdateData


In [103]:
if isFirstLoad==False:
    listModelLogObjectIDs=[]
    dfModelLog=list_model_log(last_imported,content_id)
    if dfModelLog.empty==True:
        print("No row to be imported.")
        exit()
    else:
       print("Get row imported from model log to set action") 
       dfModelLog=select_actual_action( dfModelLog)
       listModelLogObjectIDs=dfModelLog['id'].tolist()
       print(dfModelLog.info())
       print(dfModelLog)       
       print(listModelLogObjectIDs) 


    SELECT object_id, action,TO_CHAR(date_created,'YYYY-MM-DD HH24:MI:SS') as date_created FROM models_logging_change
    WHERE date_created  AT time zone 'utc' >= '2023-12-30 06:15:43' AND content_type_id = 37 
    ORDER BY object_id, date_created
    
Retrieve all rows after 2023-12-30 06:15:43
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   object_id     6 non-null      object
 1   action        6 non-null      object
 2   date_created  6 non-null      object
dtypes: object(3)
memory usage: 272.0+ bytes
None
Get row imported from model log to set action
  object_id   action         date_created
0    105979  changed  2023-12-30 13:19:31
----------------------------------------------------------------
  object_id   action         date_created
1    105980  changed  2023-12-30 13:19:31
--------------------------------------------------------------

# Load view and transform

In [104]:
def retrive_next_data_from_view(x_view,x_id,x_listModelLogObjectIDs):
    if len(x_listModelLogObjectIDs)>1:
     sql_view=f"select *  from {x_view}  where {x_id} in {tuple(x_listModelLogObjectIDs)}"
    else:
     sql_view=f"select *  from {x_view}  where {x_id} ={x_listModelLogObjectIDs[0]}"
    
    print(sql_view)
    df=list_data(sql_view,None,get_postgres_conn())

    if df.empty==True:
     return None
    df=df.drop(columns='updated_at')
    return df 


def retrive_first_data_from_view(x_view,x_last_imported):
     sql_view=f"select *  from {x_view}  where  updated_at AT time zone 'utc' >= '{x_last_imported}'"
     print(sql_view)
     df=list_data(sql_view,None,get_postgres_conn())
     if df.empty==True:
            return None
     df=df.drop(columns='updated_at')
     df['action']='added'
     return df   
     


if isFirstLoad:
 df=retrive_first_data_from_view(view_name,last_imported)
else:
 df=retrive_next_data_from_view(view_name,view_name_id,listModelLogObjectIDs)   
    
if df is None:
    print("No row to be imported.")
    exit()
    
print(df.info())
df

select *  from pmr_pm_item  where pm_item_id in (105979, 105980, 105981, 106008, 106009, 106010)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   pm_item_id         6 non-null      int64 
 1   pm_id              6 non-null      int64 
 2   is_pm              6 non-null      bool  
 3   pm_engineer        2 non-null      object
 4   actual_date        2 non-null      object
 5   document_engineer  0 non-null      object
 6   document_date      0 non-null      object
 7   inventory_id       6 non-null      int64 
 8   is_complete        6 non-null      bool  
dtypes: bool(2), int64(3), object(4)
memory usage: 476.0+ bytes
None


,pm_item_id,pm_id,is_pm,pm_engineer,actual_date,document_engineer,document_date,inventory_id,is_complete
0,105979,5248,True,Akaradej Ketruskul,2024-01-01,None,None,11563,False
1,105980,5248,True,Akaradej Ketruskul,2024-01-01,None,None,11562,False
2,105981,5248,False,None,None,None,None,12279,False
3,106008,5256,True,None,None,None,None,11562,False
4,106009,5256,True,None,None,None,None,11563,False
5,106010,5256,True,None,None,None,None,12279,False


# Data Transaformation
* IF The first load then add actio='Added'
* IF The nextload then Merge LogDF and ViewDF and add deleted row 
  * Get Deleted Items  to Create deleted dataframe by using listDeleted
  * If there is one deletd row then  we will merge it to master dataframe

In [105]:
def add_acutal_action_to_df_at_next(df,dfUpdateData):
    merged_df = pd.merge(df, dfUpdateData, left_on=view_name_id, right_on='id', how='inner')
    merged_df = merged_df.drop(columns=['id'])

    listSelected = df[view_name_id].tolist()
    print(listSelected)

    set1 = set(listModelLogObjectIDs)
    set2 = set(listSelected)
    listDeleted = list(set1.symmetric_difference(set2))

    print(listDeleted)

    if len(listDeleted)>0:
        print("There are some deleted rows")
        dfDeleted=pd.DataFrame(data=listDeleted,columns=[view_name_id])
        dfDeleted['action']='deleted'
        print(dfDeleted)
        merged_df=pd.concat([merged_df,dfDeleted],axis=0)

    else:
        print("No row deleted")

    return merged_df    

if isFirstLoad==False:
 df=add_acutal_action_to_df_at_next(df,dfModelLog)


[105979, 105980, 105981, 106008, 106009, 106010]
[]
No row deleted


# Check duplicate ID & reset index

In [106]:
hasDplicateIDs = df[view_name_id].duplicated().any()
if  hasDplicateIDs:
 raise Exception("There are some duplicate id on dfUpdateData")
else:
 print(f"There is no duplicate {view_name_id} ID")  


# merged_df['imported_at']=dt_imported
df=df.reset_index(drop=True  )
print(df.info())
print(df)

There is no duplicate pm_item_id ID
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   pm_item_id         6 non-null      int64 
 1   pm_id              6 non-null      int64 
 2   is_pm              6 non-null      bool  
 3   pm_engineer        2 non-null      object
 4   actual_date        2 non-null      object
 5   document_engineer  0 non-null      object
 6   document_date      0 non-null      object
 7   inventory_id       6 non-null      int64 
 8   is_complete        6 non-null      bool  
 9   action             6 non-null      object
dtypes: bool(2), int64(3), object(5)
memory usage: 524.0+ bytes
None
   pm_item_id  pm_id  is_pm         pm_engineer actual_date document_engineer  \
0      105979   5248   True  Akaradej Ketruskul  2024-01-01              None   
1      105980   5248   True  Akaradej Ketruskul  2024-01-01         

# Insert data to BQ data frame

In [18]:
if get_bq_table():
    try:
        insertDataFrameToBQ(df)
    except Exception as ex:
        raise ex

Table pongthorn.SMartData_Temp.temp_pm_plan already exists.
[SchemaField('pm_id', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('project_id', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('planned_date', 'DATE', 'NULLABLE', None, None, (), None), SchemaField('ended_pm_date', 'DATE', 'NULLABLE', None, None, (), None), SchemaField('pm_period', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('team_lead', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('action', 'STRING', 'NULLABLE', None, None, (), None)]
Total  4613 Imported data to pongthorn.SMartData_Temp.temp_pm_plan on bigquery successfully


In [19]:
updater["metadata"][view_name].value=dt_imported.strftime("%Y-%m-%d %H:%M:%S")
updater.update_file() 

<ConfigUpdater [
    <Section: 'metadata' [
        <Option: pmr_pm_plan = '2023-12-30 04:28:45'>
        <Option: pmr_pm_item = '2019-01-01 00:00:00'>
        <Option: pmr_project = '2019-01-01 00:00:00'>
        <Option: pmr_inventory = '2019-01-01 00:00:00'>
    ]>
]>

In [20]:
print(datetime.now(timezone.utc) )

2023-12-30 04:29:29.908823+00:00
